In [1]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import numpy as np
import cv2
import os
from skimage.morphology import convex_hull_image, binary_closing, square, dilation
from scipy import ndimage

In [2]:
#  load all images from a directory
def load_images_from_folder(folder):
    images = []
    filenames = []
    for filename in tqdm(os.listdir(folder)):
        try:
            img = Image.open(os.path.join(folder,filename))
            if img is not None:
                if img.mode != 'RGB':
                    img = img.convert('RGB')
                img = np.array(img)
                images.append(img)
                filenames.append(filename)
        except Exception as e:
            print(e)
    return images, filenames

In [3]:
# load all jsons from a directory
def load_jsons_from_folder(folder):
    jsons = []
    filenames = []
    for filename in tqdm(os.listdir(folder)):
        try:
            json = pd.read_json(os.path.join(folder,filename))
            if json is not None:
                jsons.append(json)
                filenames.append(filename)
        except Exception as e:
            print(e)
    return jsons, filenames

In [4]:
LOAD_CLEAN = True
if LOAD_CLEAN:
    human_parsing = load_images_from_folder('Task/dataset/human_parsing')
    image = load_images_from_folder('Task/dataset/image')
    pose_json = load_jsons_from_folder('Task/dataset/pose_json')
    pose_img = load_images_from_folder('Task/dataset/pose_img')
    df = pd.DataFrame()
    df['human_parsing'] = human_parsing[0]
    df['image'] = image[0]
    df['pose_json'] = pose_json[0]
    df['pose_img'] = pose_img[0]
    df['ids'] = [a.split('.')[0] for a in image[1]]
    df['pose_keypoints'] = df.pose_json.apply(lambda x: x.people[0].get('pose_keypoints_2d'))
    left_ankles = [a[3*11:3*12-1] for a in tqdm(df.pose_keypoints)]
    right_ankles = [a[3*14:3*15-1] for a in tqdm(df.pose_keypoints)]
    df['left_ankles'] = left_ankles
    df['right_ankles'] = right_ankles
    df.to_pickle('df.pkl')
else:
    df = pd.read_pickle('df.pkl')

  1%|▏         | 10/705 [00:00<00:13, 50.30it/s]

cannot identify image file 'Task/dataset/image\\.DS_Store'


100%|██████████| 704/704 [00:00<00:00, 705565.12it/s]


In [ ]:
short_colors = {(64, 0, 128), (192, 0, 0)}
legs_colors = {(128,64,0), (0,64,0)}

shirt_colors = {(128,0,128), (128,128,128), (128,64,128)}
hands_colors = {(192,128,128),(64,128,128)}

# skirt_colors = short_colors.union(legs_colors)
# body_colors = shirt_colors.union(hands_colors)

In [ ]:
def get_color_mask(img, colors):
    mask = np.zeros(img.shape[:2], dtype=np.uint8)
    for color in colors:
        mask += np.all(img == color, axis=-1).astype(np.uint8)
    return mask

def clean_outliers(mask):
    labels, num_labels = ndimage.label(mask)
    component_sizes = ndimage.sum(mask, labels, range(num_labels + 1))
    mask_largest_component = (labels == component_sizes.argmax())
    return mask_largest_component

def apply_mask(img, mask):
    img = np.array(img)
    img[mask] = (128, 128, 128)
    return Image.fromarray(img)

In [ ]:
RENDER = True
if RENDER:
    tqdm._instances.clear()
    mask_shorts = [get_color_mask(a, short_colors) for a in tqdm(human_parsing[0])]
    mask_legs = [get_color_mask(a, legs_colors) for a in tqdm(human_parsing[0])]
    mask_legs_and_shorts = [np.logical_or(a, b) for a, b in tqdm(zip(mask_shorts, mask_legs))]

    mask_shirts = [get_color_mask(a, shirt_colors) for a in tqdm(human_parsing[0])]
    mask_hands = [get_color_mask(a, hands_colors) for a in tqdm(human_parsing[0])]
    mask_upper_body = [np.logical_or(a, b) for a, b in tqdm(zip(mask_shirts, mask_hands))]
    mask_upper_body_clean = [clean_outliers(a) for a in tqdm(mask_upper_body)]

    mask_skirts_clean = [np.logical_and(a, np.logical_not(b)) for a, b in tqdm(zip(mask_legs_and_shorts, mask_upper_body_clean))]
    mask_skirts_clean_no_outliers = [clean_outliers(a) for a in tqdm(mask_skirts_clean)]
    mask_skirts_clean_convex = [convex_hull_image(a) for a in tqdm(mask_skirts_clean_no_outliers)]
    mask_skirts_final = [np.logical_and(a, np.logical_not(b)) for a, b in tqdm(zip(mask_skirts_clean_convex, mask_upper_body_clean))]

    processed_images = [apply_mask(a, b) for a, b in tqdm(zip(df.image, mask_skirts_final))]
    
    df['mask_shorts'] = mask_shorts
    df['mask_legs'] = mask_legs
    df['mask_shirts'] = mask_shirts
    df['mask_hands'] = mask_hands

    df['mask_legs_and_shorts'] = mask_legs_and_shorts
    df['mask_upper_body'] = mask_upper_body
    df['mask_skirts_clean'] = mask_skirts_clean
    df['mask_skirts_clean_no_outliers'] = mask_skirts_clean_no_outliers
    df['mask_skirts_clean_convex'] = mask_skirts_clean_convex
    df['mask_skirts_final'] = mask_skirts_final
    df['processed_images'] = processed_images
    df.to_pickle('df_full.pkl')
else:
    df = pd.read_pickle('df_full.pkl')

In [ ]:
# save processed_images to /preprocessed
SAVE_PROCESSED = False
if SAVE_PROCESSED:
    for i, img in tqdm(enumerate(processed_images)):
        img.save('C:/code/python/test_task_big/preprocessed_5/' + df.ids[i] + '.jpg')

In [ ]:
left_ankles = [a[3*11:3*12-1] for a in tqdm(df.pose_keypoints)]
right_ankles = [a[3*14:3*15-1] for a in tqdm(df.pose_keypoints)]
left_ankles[0], right_ankles[0]

In [ ]:
# make mask over df.image[0] such there is line between left and right ankle
# and everything below this line is black
# and everything above this line is white

In [ ]:
img_cols = set(df.columns) - set(['ids', 'pose_keypoints', 'pose_json'])
img_cols = [
    'image',
    'human_parsing',
    'pose_img',
    'mask_shorts',
    'mask_legs',
    'mask_legs_and_shorts',
    'mask_shirts',
    'mask_hands',
    'mask_upper_body',
    'mask_skirts_clean',
    'mask_skirts_clean_no_outliers',
    'mask_skirts_clean_convex',
    'mask_skirts_final',
    'processed_images'
 ]

In [ ]:
def show_images_df(df, cols, index):
    fig, axes = plt.subplots(1, len(cols), figsize=(20, 20))
    for i, col in enumerate(cols):
        axes[i].imshow(df[col][index])
        axes[i].set_title(col)
    plt.show()
    # save image
    fig.savefig('full_outs/' + df.ids[index] + '.jpg')

In [ ]:
show_images_df(df, img_cols, 0)

In [ ]:
# HELPER FUNCTIONS
# def draw_pose(img, poses):
#     draw = ImageDraw.Draw(img)
#     for i in range(0, len(poses), 3):
#         draw.ellipse((poses[i]-5, poses[i+1]-5, poses[i]+5, poses[i+1]+5), fill=(255, 0, 0, 255))
#         draw.text((poses[i], poses[i+1]), str(i//3), fill=(255, 0, 0, 0), font= ImageFont.truetype("arial.ttf", 30))
#     return img
# draw_pose(df.image[0], df.pose_keypoints[0])

